<a href="https://colab.research.google.com/github/FatimaSenan/Tps_NLP/blob/main/Tp3_Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
files = files.upload()

Saving movie_review.csv to movie_review.csv


In [29]:
import pandas as pd
df=pd.read_csv('/content/movie_review.csv')
print(df.head())

   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  
0  films adapted from comic books have had plenty...  pos  
1  for starters , it was created by alan moore ( ...  pos  
2  to say moore and campbell thoroughly researche...  pos  
3  the book ( or " graphic novel , " if you will ...  pos  
4  in other words , don't dismiss this film becau...  pos  


# Pre-processing des données textuelles

In [31]:
#mise en miniscule
df['text'] = df['text'].str.lower()
print(df['text'])

0        films adapted from comic books have had plenty...
1        for starters , it was created by alan moore ( ...
2        to say moore and campbell thoroughly researche...
3        the book ( or " graphic novel , " if you will ...
4        in other words , don't dismiss this film becau...
                               ...                        
64715    that lack of inspiration can be traced back to...
64716    like too many of the skits on the current inca...
64717    after watching one of the " roxbury " skits on...
64718     bump unsuspecting women , and . . . that's all .
64719    after watching _a_night_at_the_roxbury_ , you'...
Name: text, Length: 64720, dtype: object


In [32]:
# Suppression des ponctuations
import re
def no_ponctuation(text):
  text = re.sub(r'[^\w\s]', '', text)
  return text

df['text'] = df['text'].apply(no_ponctuation)
print(df['text'])

0        films adapted from comic books have had plenty...
1        for starters  it was created by alan moore  an...
2        to say moore and campbell thoroughly researche...
3        the book  or  graphic novel   if you will  is ...
4        in other words  dont dismiss this film because...
                               ...                        
64715    that lack of inspiration can be traced back to...
64716    like too many of the skits on the current inca...
64717    after watching one of the  roxbury  skits on s...
64718           bump unsuspecting women  and    thats all 
64719    after watching _a_night_at_the_roxbury_  youll...
Name: text, Length: 64720, dtype: object


In [36]:
from gensim.parsing.preprocessing import remove_stopwords

df['text'] = df['text'].apply(remove_stopwords)


print(df.head())

   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  
0  films adapted comic books plenty success theyr...  pos  
1  starters created alan moore eddie campbell bro...  pos  
2  moore campbell thoroughly researched subject j...  pos  
3  book graphic novel 500 pages long includes nea...  pos  
4                     words dont dismiss film source  pos  


# Entraînement du modèle Word2Vec :

In [38]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
# Tokenization
df['text'] = df['text'].apply(word_tokenize)

# modèle Word2Vec
model = Word2Vec(df['text'], vector_size=100, window=5, min_count=1)


# Vectorisation des reviews de movies

In [40]:
import numpy as np
def get_review_vector(review, model):
    review_vector = np.zeros(model.vector_size)
    count = 0
    for word in review:
        if word in model.wv.key_to_index:
            review_vector += model.wv[word]
            count += 1
    if count != 0:
        review_vector /= count
    return review_vector
df['review_vector'] = df['text'].apply(lambda x: get_review_vector(x, model))
print(df.head())


   fold_id cv_tag  html_id  sent_id  \
0        0  cv000    29590        0   
1        0  cv000    29590        1   
2        0  cv000    29590        2   
3        0  cv000    29590        3   
4        0  cv000    29590        4   

                                                text  tag  \
0  [films, adapted, comic, books, plenty, success...  pos   
1  [starters, created, alan, moore, eddie, campbe...  pos   
2  [moore, campbell, thoroughly, researched, subj...  pos   
3  [book, graphic, novel, 500, pages, long, inclu...  pos   
4               [words, dont, dismiss, film, source]  pos   

                                       review_vector  
0  [-0.11405561272176387, 0.5674848140257857, 0.2...  
1  [0.00986517680939869, 0.37984744837740436, 0.2...  
2  [-0.1202960463357158, 0.545409174123779, 0.279...  
3  [-0.10097842430695891, 0.48074913736094127, 0....  
4  [-0.21565992757678032, 0.7580236673355103, 0.3...  


# Division des données :

In [43]:
from sklearn.model_selection import train_test_split
# ensembles d'entraînement et de test
X = np.array(df['review_vector'].tolist())
y = np.array(df['tag'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Construction d&#39;un classificateur :

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
#Columns pour la classification : Text (features) et Tag (label)
X = np.array(df['text'].tolist())
y = np.array(df['tag'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)




# Construction d'un classificateur :

In [44]:
from sklearn.linear_model import LogisticRegression
# Initialiser le modèle
model = LogisticRegression(max_iter=1000)
# Entraîner le modèle
model.fit(X_train, y_train)

# Performances du modèle

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# les prédictions du modèle
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, pos_label='pos')
recall = recall_score(y_test, y_pred, pos_label='pos')
f1 = f1_score(y_test, y_pred, pos_label='pos')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("score:", f1)

Accuracy: 0.5700710754017305
Precision: 0.5573198604595212
Recall: 0.7108008591592513
score: 0.6247724361135459
